In [1]:
import pandas as pd
import os
import sys
from pathlib import Path

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
root_path = (Path(".")).resolve()
root_path

PosixPath('/Users/belen/Documents/Data science/Courses/Udacity/Data science nanodegree/Projects/AirBnb data analysis')

In [3]:
sys.path.append(str(root_path.resolve()))

In [7]:
data_path = root_path / "data" / "raw"
data_path

PosixPath('/Users/belen/Documents/Data science/Courses/Udacity/Data science nanodegree/Projects/AirBnb data analysis/data/raw')

In [5]:
data_path

PosixPath('/Users/belen/Documents/Data science/Courses/Udacity/Data science nanodegree/Projects/AirBnb data analysis/data/raw')

In [6]:
data_files = list(data_path.glob("*.*"))
list(x.name for x in sorted(data_files, key=lambda x: x.stem))

[]